In [34]:
from sklearn.ensemble import GradientBoostingRegressor
# 导入均方误差计算
from sklearn.metrics import mean_squared_error
# 导入标准化模块包
from sklearn.preprocessing import RobustScaler
# 导入划分数据集包，交叉验证包
from sklearn.model_selection import train_test_split,KFold,cross_val_score
# 导入Xgboost算法包
import xgboost as xgb
from  sklearn import datasets 
import pandas as pd 
import xgboost as xgb
import numpy as np
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing  
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV

from sklearn.compose import ColumnTransformer  
from sklearn.preprocessing import OneHotEncoder, StandardScaler  

In [35]:
dataset_df = pd.read_csv('./spaceship-titanic/train.csv')
test_df = pd.read_csv('./spaceship-titanic/test.csv')

In [36]:
# dataset_df.info
# dataset_df.loc[dataset_df['Transported'] == False, 'Transported'] = 0
# dataset_df.loc[dataset_df['Transported'] == True, 'Transported'] = 1

y = dataset_df.pop('Transported')

In [37]:
all_data = pd.concat([dataset_df, test_df], axis=0, ignore_index=True)

In [38]:
all_data.loc[all_data['CryoSleep'] == False, 'CryoSleep'] = 0
all_data.loc[all_data['CryoSleep'] == True, 'CryoSleep'] = 1



In [39]:
all_data.loc[all_data['VIP'] == False, 'VIP'] = 0
all_data.loc[all_data['VIP'] == True, 'VIP'] = 1

all_data

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0001_01,Europa,0,B/0/P,TRAPPIST-1e,39.0,0,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy
1,0002_01,Earth,0,F/0/S,TRAPPIST-1e,24.0,0,109.0,9.0,25.0,549.0,44.0,Juanna Vines
2,0003_01,Europa,0,A/0/S,TRAPPIST-1e,58.0,1,43.0,3576.0,0.0,6715.0,49.0,Altark Susent
3,0003_02,Europa,0,A/0/S,TRAPPIST-1e,33.0,0,0.0,1283.0,371.0,3329.0,193.0,Solam Susent
4,0004_01,Earth,0,F/1/S,TRAPPIST-1e,16.0,0,303.0,70.0,151.0,565.0,2.0,Willy Santantines
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12965,9266_02,Earth,1,G/1496/S,TRAPPIST-1e,34.0,0,0.0,0.0,0.0,0.0,0.0,Jeron Peter
12966,9269_01,Earth,0,NaN,TRAPPIST-1e,42.0,0,0.0,847.0,17.0,10.0,144.0,Matty Scheron
12967,9271_01,Mars,1,D/296/P,55 Cancri e,NaN,0,0.0,0.0,0.0,0.0,0.0,Jayrin Pore
12968,9273_01,Europa,0,D/297/P,NaN,NaN,0,0.0,2680.0,0.0,0.0,523.0,Kitakan Conale


In [40]:
# mode_value = all_data['HomePlanet'].mode()[0]  # 获取众数  


all_data['HomePlanet'].fillna(all_data['HomePlanet'].mode()[0], inplace=True)
all_data['CryoSleep'].fillna(all_data['CryoSleep'].mode()[0], inplace=True)
all_data['Cabin'].fillna(all_data['Cabin'].mode()[0], inplace=True)
all_data['Destination'].fillna(all_data['Destination'].mode()[0], inplace=True)

all_data['Age'].fillna(all_data['Age'].median(), inplace=True)

all_data['ShoppingMall'].fillna(all_data['ShoppingMall'].median(), inplace=True)

# all_data[['Cabin_1', 'Cabin_2', 'Cabin_3']] = all_data['Cabin'].str.split('/', expand=True)




In [41]:
all_data[['Cabin_1', 'Cabin_2', 'Cabin_3']] = all_data['Cabin'].str.split('/', expand=True)

In [42]:
Id = all_data.pop('PassengerId')


In [43]:
# all_data = pd.get_dummies(all_data)
num_cols = [cname for cname in all_data.columns if all_data[cname].dtype in ['int64', 'float64']]
cat_cols = [cname for cname in all_data.columns if all_data[cname].dtype == "object"]

le = LabelEncoder()  

for col in cat_cols:
    encoded_column = le.fit_transform(all_data[col])  
    all_data[col] = encoded_column

train = all_data.loc[all_data.index < 8693]
test = all_data.loc[all_data.index >= 8693]


# train = all_data[all_data[''] < 1461]
# train.pop('Id')

# test = all_data[all_data['Id'] >= 1461]
# Id = test.pop('Id')

In [44]:
train
y

0       False
1        True
2       False
3       False
4        True
        ...  
8688    False
8689    False
8690     True
8691    False
8692     True
Name: Transported, Length: 8693, dtype: bool

In [58]:
cv_params = {'learning_rate': [0.01, 0.02, 0.03]}
other_params = {'learning_rate': 0.03, 'n_estimators': 400, 'max_depth': 3, 'min_child_weight': 2, 'seed': 0,
                    'subsample': 0.6, 'colsample_bytree': 0.6, 'gamma': 0.2, 'reg_alpha': 3, 'reg_lambda': 1, 'objective': 'multi:softmax', 'num_class':2}

model = xgb.XGBClassifier(**other_params)
optimized_GBM = GridSearchCV(estimator=model, param_grid=cv_params, scoring='roc_auc', cv=5, verbose=1, n_jobs=4)

optimized_GBM.fit(train, y)
evalute_result = optimized_GBM.cv_results_['mean_test_score']
print('每轮迭代运行结果:{0}'.format(evalute_result))
print('参数的最佳取值：{0}'.format(optimized_GBM.best_params_))
print('最佳模型得分:{0}'.format(optimized_GBM.best_score_))


print(optimized_GBM.best_estimator_)

Fitting 5 folds for each of 3 candidates, totalling 15 fits
每轮迭代运行结果:[0.8707689  0.87770442 0.87932816]
参数的最佳取值：{'learning_rate': 0.03}
最佳模型得分:0.8793281580433249
XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.6, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=0.2, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.03, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=3, max_leaves=None,
              min_child_weight=2, missing=nan, monotone_constraints=None,
              n_estimators=400, n_jobs=None, num_class=2,
              num_parallel_tree=None, objective='multi:softmax', ...)


In [59]:
Id[Id.index >= 8693]
pred = optimized_GBM.best_estimator_.predict(test)

out = {
    'PassengerId':Id[Id.index >= 8693],
    'Transported':pred
}

out = pd.DataFrame(out)

out.loc[out['Transported'] == 0, 'Transported'] = False
out.loc[out['Transported'] == 1, 'Transported'] = True
print(out)

out.to_csv('submisson_space.csv',index=False)

      PassengerId Transported
8693      0013_01        True
8694      0018_01       False
8695      0019_01        True
8696      0021_01        True
8697      0023_01        True
...           ...         ...
12965     9266_02        True
12966     9269_01       False
12967     9271_01        True
12968     9273_01        True
12969     9277_01        True

[4277 rows x 2 columns]
